In [26]:
import os
import re
import uuid
from pathlib import Path

In [18]:
%load_ext autoreload

In [19]:
%autoreload 2

In [20]:
import wombat.bsub as bsub

In [27]:
def map_host_command(h='host'):
    return f'export LSF_DOCKER_NETWORK={h}'

def map_volumes_command(dirs):
    vol_str = ' '.join(
        [f'{d}:{d}' for d in dirs]
    )
    return f'export LSF_DOCKER_VOLUMES="{vol_str}"'

In [3]:
dirs = [
    '/path/to/here', 'path/to/there'
]
map_volumes(dirs)

'export LSF_DOCKER_VOLUMES="/path/to/here:/path/to/here path/to/there:path/to/there"'

In [13]:
def create_job_group_command(group_name,  n=10, username='estorrs'):
    return f'bgadd -L {n} /{username}/{group_name}'
    
def job_group_status_command(group_name, username='estorrs'):
    return f'bjgroup -s /{username}/{group_name}'

In [14]:
create_job_group_command('test_group')

'bgadd -L 10 /estorrs/test_group'

bsub -R 'select[mem>100000] rusage[mem=100000] span[hosts=1]' -M 30000000 -q dinglab -G compute-dinglab -o job.output -oo job.log -a 'docker(python:3.7)' 'bash blah.sh`

bsub -G ${group_name} -q foo-condo -R 'rusage[mem=8GB]' -M 6GB -a 'docker(alpine)' /bin/true

bsub -R 'select[mem>100000] rusage[mem=100000] span[hosts=1]' -M 100000000 -Is -G compute-dinglab -q dinglab-interactive -a 'docker(python:3.7)' /bin/bash

In [5]:
# bsub -g /${compute_username}/${group_name} -J ${job_name}

In [87]:
def bsub_command(command='/bin/bash', mem=10, max_mem=None, docker='python:3.8', queue='dinglab',
                        group='compute-dinglab', group_name=None, job_name=None, interactive=False,
                        username='estorrs', log_fp=None):
    
    if max_mem is None:
        max_mem = mem
        
    if interactive and queue != f'{queue}-interactive':
        queue = f'{queue}-interactive'
    
    base = f'bsub'
    if mem is not None:
        base += ' -R \'rusage[mem={mem}GB]\' -M {max_mem}GB'
        
    if queue is not None:
        base += ' -q {queue}'
    
    if group is not None:
        base += ' -G {group}'
    
    if docker is not None:
        base += f' -a \'docker({docker})\''
    
    if group_name is not None:
        job_name = str(uuid.UUID4()) if job_name is None else job_name
        base += f' -g /{username}/{group_name} -J {job_name}'
        
    if log_fp is not None:
        base += f' -oo {log_fp}'
        
    if interactive:
        base += ' -Is'
    
    if command is not None:
        base += f' \'{command}\''
    
    return base
        
    

In [86]:
bsub_command()

"bsub -R 'rusage[mem={mem}GB]' -M {max_mem}GB -q {queue} -G {group} -a 'docker(python:3.8)' '/bin/bash'"

In [17]:
bsub_command(interactive=True)

"bsub -R 'rusage[mem=10GB]' -M 10GB -q dinglab-interactive -G compute-dinglab -a 'docker(python:3.8)' -Is '/bin/bash'"

In [21]:
bsub.bsub_command(interactive=True)

"bsub -R 'rusage[mem=10GB]' -M 10GB -q dinglab-interactive -G compute-dinglab -a 'docker(python:3.8)' -Is '/bin/bash'"

In [25]:
args = bsub.DEFAULT_ARGS
args

{'mem': 10,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': None,
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [35]:
commands = ['/bin/bash hello wooo', '/bin/bash hello bbb']
job_names = ['wo', 'bb']
volumes = ['/path/to/dir1', '/path/to/dir2']
log_dir = '/path/to/log_dir'
args['group_name'] = 'test_group'

In [60]:
def housekeeping_priors(log_dir, args, volumes=None):
    if volumes is None:
        volumes = []
    
    try:
        Path(log_dir).mkdir(parents=True, exist_ok=True)
    except:
        print(f'Failed to create log dir {log_dir}')

    if log_dir not in volumes:
        volumes.append(log_dir)
        
    if volumes:
        mv_command = map_volumes_command(volumes)
    else:
        mv_command = None
    
    if args['group_name'] is not None:
        jg_command = create_job_group_command(
            args['group_name'],  n=args['n_concurrent'], username=args['username'])
    else:
        jg_command = None
        
    return mv_command, jg_command

def batch_bsub_commands(commands, job_names, log_dir, args, volumes=None):
    mv_command, jg_command = housekeeping_priors(log_dir, args, volumes=volumes)
            
    bsub_commands = []
    for command, job_name in zip(commands, job_names):
        log_fp = os.path.join(log_dir, f'{job_name}.txt')

        c = bsub_command(
            command=command, mem=args['mem'], max_mem=args['max_mem'],
            docker=args['docker'], queue=args['queue'], group=args['group'],
            group_name=args['group_name'], job_name=job_name, interactive=args['interactive'],
            log_fp=log_fp, username=args['username'])

        bsub_commands.append(c)
        
    all_commands = [c for c in [mv_command, jg_command]
                    if c is not None]
    all_commands += bsub_commands
        
    return all_commands

In [61]:
batch_bsub_commands(commands, job_names, log_dir, args, volumes=volumes)

Failed to create log dir /path/to/log_dir


['export LSF_DOCKER_VOLUMES="/path/to/dir1:/path/to/dir1 /path/to/dir2:/path/to/dir2 /path/to/log_dir:/path/to/log_dir"',
 'bgadd -L 10 /estorrs/test_group',
 "bsub -R 'rusage[mem=10GB]' -M 10GB -q dinglab -G compute-dinglab -a 'docker(python:3.8)' -g /estorrs/test_group -J wo -oo /path/to/log_dir/wo.txt '/bin/bash hello wooo'",
 "bsub -R 'rusage[mem=10GB]' -M 10GB -q dinglab -G compute-dinglab -a 'docker(python:3.8)' -g /estorrs/test_group -J bb -oo /path/to/log_dir/bb.txt '/bin/bash hello bbb'"]

In [78]:
dconfig = 'dat/cromwell-config-db.compute1.dat'
cwl_fp = '../../cwl/workflows/tindaisy2.cwl'
inputs_fp = 'dat/C3N-02996.BIOTEXT_YEyWzRW.yaml'
volumes = ['/path/to/dira', '/path/to/dir/b']
workflow_dir = '/path/to/w_dir'


bsub -q dinglab-interactive -Is -a "docker(mwyczalkowski/cromwell-runner)" 

/usr/bin/java -Dconfig.file=dat/cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i dat/C3N-02996.BIOTEXT_YEyWzRW.yaml ../../cwl/workflows/tindaisy2.cwl


In [99]:
def submit_cwl_command(dconfig, cwl_fp, inputs_fp, java='/usr/bin/java', jar='/usr/local/cromwell/cromwell-47.jar'):
    cmd = f'{java} -Dconfig.file={dconfig}'
    cmd += ' -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore'
    cmd += f' -jar {jar} -t cwl -i {inputs_fp} {cwl_fp}'
    return cmd

def cromwell_commands(dconfig, cwl_fp, inputs_fp, workflow_dir, args, volumes):
    mv_command, jg_command = housekeeping_priors(log_dir, args, volumes=volumes)
    mh_command = map_host_command()
    source_lsf_command = 'source /opt/ibm/lsfsuite/lsf/conf/lsf.conf'
    
    start_server_command = bsub_command(
            command=None, group=None, mem=None,
            docker='mwyczalkowski/cromwell-runner', queue=args['queue'], interactive=True)
    
    submit_command = submit_cwl_command(dconfig, cwl_fp, inputs_fp)
    
    all_commands = [c for c in [source_lsf_command, mh_command, mv_command, jg_command, start_server_command, submit_command]
                    if c is not None]
    return all_commands

In [93]:
cromwell_commands(dconfig, cwl_fp, inputs_fp, workflow_dir, args, volumes)

Failed to create log dir /path/to/log_dir


['export LSF_DOCKER_NETWORK=host',
 'export LSF_DOCKER_VOLUMES="/path/to/dira:/path/to/dira /path/to/dir/b:/path/to/dir/b /path/to/log_dir:/path/to/log_dir"',
 'bgadd -L 10 /estorrs/test_group',
 "bsub -q {queue} -a 'docker(mwyczalkowski/cromwell-runner)' -Is",
 '/usr/bin/java -Dconfig.file=dat/cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar -t cwl -i dat/C3N-02996.BIOTEXT_YEyWzRW.yaml ../../cwl/workflows/tindaisy2.cwl']

In [102]:
bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, workflow_dir, args, volumes)

['source /opt/ibm/lsfsuite/lsf/conf/lsf.conf',
 'export LSF_DOCKER_NETWORK=host',
 'export LSF_DOCKER_VOLUMES="/path/to/dira:/path/to/dira /path/to/dir/b:/path/to/dir/b /path/to/log_dir:/path/to/log_dir"',
 'bgadd -L 10 /estorrs/test_group',
 "bsub -q dinglab-interactive -a 'docker(mwyczalkowski/cromwell-runner)' -Is",
 '/usr/bin/java -Dconfig.file=dat/cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar -t cwl -i dat/C3N-02996.BIOTEXT_YEyWzRW.yaml ../../cwl/workflows/tindaisy2.cwl']

In [103]:
filepath = '../sandbox/run_cromwell.sh'
bsub.write_command_file(
    bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, workflow_dir, args, volumes),
    filepath)